# Metrics
To compare two datasets (e.g. brain recordings and model predictions), we use *metrics* that compute a similarity score.

### Pre-defined metrics

Brain-Score comes with many standard metrics used in the field.
For instance, the *neural predictivity metric* (here with `ridge` regression)
(1) uses linear regression to map between two matrices (e.g. from model activations to neural firing rates),
(2) computes the correlation between predicted and actual firing rates on held-out images, and
(3) cross-validates 1 and 2 to estimate a generalization error.

In [1]:
from brainscore_vision import load_metric

metric = load_metric('ridge')

All metrics have a `__call__` method which takes as input two sets of measurements and produces a score.
See e.g. below for the result of running the `ridge` neural predictivity metric on "two" dummy datasets.

In [2]:
import numpy as np
from numpy.random import RandomState

from brainio.assemblies import NeuroidAssembly

rnd = RandomState(0)  # seed for reproducibility
assembly = NeuroidAssembly((np.arange(30 * 25) + rnd.standard_normal(30 * 25)).reshape((30, 25)),
                           coords={'stimulus_id': ('presentation', np.arange(30)),
                                   'object_name': ('presentation', ['a', 'b', 'c'] * 10),
                                   'neuroid_id': ('neuroid', np.arange(25)),
                                   'region': ('neuroid', ['V1'] * 25)},
                           dims=['presentation', 'neuroid'])
source, target = assembly, assembly  # we're testing how well the metric can predict the dataset itself
score = metric(source=source, target=target)
print(score)

cross-validation: 100%|██████████| 10/10 [00:00<00:00, 14.42it/s]

<xarray.Score ()>
array(0.99999893)
Attributes:
    raw:      <xarray.Score (split: 10, neuroid: 25)>\narray([[0.99999998, 0....
    error:    <xarray.Score ()>\narray(7.31630414e-07)


The overall result here is a score close to 1. Note that this aggregate score averages over cross-validation splits as well as neural sites (`neuroids`).
We can also check the raw values, i.e. the value per split and per neuroid.

In [3]:
print(score.raw)

<xarray.Score (split: 10, neuroid: 25)>
array([[0.99999998, 0.99999989, 0.99999983, 1.        , 0.99999891,
        0.99999917, 0.99999993, 0.99999956, 0.99999942, 0.99999974,
        0.99999999, 0.99999915, 0.99999879, 0.9999982 , 0.99999911,
        0.99999877, 0.99999991, 0.9999998 , 0.99999998, 0.99999958,
        0.99999993, 0.99999995, 1.        , 0.99999968, 0.99999995],
       [1.        , 0.99999993, 0.99999997, 0.99999814, 0.9999966 ,
        0.99999999, 0.99999991, 0.99999896, 0.99999927, 0.9999978 ,
        0.99999992, 0.99999877, 0.9999997 , 0.99999992, 0.99999817,
        1.        , 0.99999991, 0.99999991, 1.        , 0.99999989,
        0.99999844, 0.99999981, 0.99999797, 0.99999852, 0.99999998],
       [0.99999995, 0.99999718, 0.99999993, 0.99999825, 0.99999958,
        0.9999935 , 0.99999814, 0.99999991, 0.99999363, 0.99999946,
        0.99999964, 0.99999453, 0.99999994, 0.9999985 , 0.99999443,
        0.99999939, 0.99999995, 0.99999222, 0.99999255, 0.99999937,
      

#### RDM

The RDM metric is another commonly used method which does not require any fitting.

In [4]:
from brainscore_vision import load_metric

rdm_metric = load_metric('rdm')
rdm_score = rdm_metric(assembly1=assembly, assembly2=assembly)
print(rdm_score)

<xarray.Score ()>
array(1.)


In practice, we recommend using the `rdm_cv` metric which includes cross-validation to estimate an error:

In [5]:
rdm_cv = load_metric('rdm_cv')
rdm_cv_score = rdm_cv(assembly1=assembly, assembly2=assembly)
print(rdm_cv_score)

cross-validation: 100%|██████████| 10/10 [00:00<00:00, 39.78it/s]

<xarray.Score ()>
array(1.)
Attributes:
    raw:      <xarray.Score (split: 10)>\narray([1., 1., 1., 1., 1., 1., 1., ...
    error:    <xarray.Score ()>\narray(0.)


### Custom metrics

It is relatively straight-forward to define your own metrics. All that is required is to implement the [`Metric` interface](https://brain-score-core.readthedocs.io/en/latest/modules/metrics.html#brainscore_core.metrics.Metric). Specifically, every metric needs to implement a `__call__` method which accepts two data assemblies and outputs a [`Score`](https://brain-score-core.readthedocs.io/en/latest/modules/metrics.html#brainscore_core.metrics.Score).

For instance, here is a metric that computes the Euclidean distance of regressed and target neural sites.

In [6]:
from brainscore_core import Metric, Score
from brainio.assemblies import DataAssembly


class DistanceMetric(Metric):
    def __call__(self, source: DataAssembly, target: DataAssembly) -> Score:
        # ensure metadata is aligned
        source = source.sortby('stimulus_id').sortby('neuroid_id')
        target = target.sortby('stimulus_id').sortby('neuroid_id')
        assert (source['stimulus_id'].values == target['stimulus_id'].values).all()
        assert (source['neuroid_id'].values == target['neuroid_id'].values).all()
        # compute distance
        difference = np.abs(target.values - source.values)  # lower is better
        score = Score(difference, coords=target.coords, dims=target.dims)  # include metadata
        score = score.mean('presentation').median('neuroid')  # aggregate over stimuli and neural sites
        score = 1 - score  # invert so that higher is better (note that we would also need to define a floor so that the score is ∈[0,1]
        return score


metric = DistanceMetric()
score = metric(assembly, assembly)
print(score)

<xarray.Score ()>
array(1.)
